In [27]:
import random
import torch
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt
from torch import nn ,optim, Tensor
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
import pandas as pd
from torch_geometric.utils import structured_negative_sampling, degree
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [28]:
movie_path = '../data/ml-latest-small/movies.csv'
rating_path = '../data/ml-latest-small/ratings.csv'
print(pd.read_csv(movie_path).head())
print(pd.read_csv(rating_path).head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [29]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [30]:
from sentence_transformers import SentenceTransformer
class SequenceEncoder:
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [31]:
class GenresEncoder:
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x

In [32]:
movie_x, movie_mapping = load_node_csv(
    movie_path, index_col='movieId', encoders={
        'title': SequenceEncoder(),
        'genres': GenresEncoder()
    })

Batches: 100%|██████████| 305/305 [00:04<00:00, 75.53it/s]


In [33]:
user_x, user_mapping = load_node_csv(rating_path, index_col='userId')

In [34]:
from torch_geometric.data import HeteroData
data = HeteroData()
data['users'].num_nodes = len(user_mapping)  # Users do not have any features.
data['items'].x = movie_x

In [9]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [10]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [ ]:
edge_index, edge_label = load_edge_csv(
    rating_path,
    src_index_col='movieId',
    src_mapping=movie_mapping,
    dst_index_col='userId',
    dst_mapping=user_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)

data['user', 'rates', 'items'].edge_index = edge_index
data['user', 'rates', 'items'].edge_label = edge_label
print(data)

In [12]:
num_node = len(user_mapping)+len(movie_mapping)
user_emb = nn.Embedding(num_embeddings=len(user_mapping)+1, embedding_dim=movie_x.size(dim=1))
nn.init.normal_(user_emb.weight, std=0.1)
user_x=torch.arange(1,len(user_mapping)+1).view(1,-1)
user_x = torch.LongTensor(user_x)
user_x = user_emb(user_x)
user_x = user_x.squeeze(0)
data['users'].x = user_x
nn_data = HeteroData()
nn_data['tgt'] = data['users'].x
nn_data['src'] = data['items'].x
nn_data['adj'] = data['user', 'rates', 'items'].edge_index



In [13]:
def sample_mini_batch(batch_size, edge_index):
    """
    Args:
        batch_size (int): 批大小
        edge_index (torch.Tensor): 2*N的边列表
    Returns:
        tuple: user indices, positive item indices, negative item indices
    """
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

In [14]:
class GNN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GNN, self).__init__(aggr='sum', flow='source_to_target')
        self.linear = torch.nn.Linear(in_channels, out_channels)
        self.sigmoid = torch.nn.Sigmoid()
        self.matrix = nn.Parameter(torch.randn(in_channels, in_channels, requires_grad=True))

    def forward(self, data: HeteroData):
        x_src, x_tgt, edge_index = data['src'], data['tgt'], data['adj']
        out = self.propagate(x_src=x_src, x_tgt=x_tgt,edge_index=edge_index)
        #out = self.linear(out)
        return out
    
    def message(self, x_src,edge_index):
        message_src = x_src[edge_index[0]]
        return message_src
    
    def aggregate(self, inputs, edge_index, dim_size=None):
        weights = self.calculate_weights(edge_index[1])
        weights = weights.view(-1, 1).expand_as(inputs)
        inputs = inputs * weights

        return super().aggregate(inputs, edge_index[1])
    

    def update(self, aggr_out):
        aggr_out = torch.matmul(aggr_out, self.matrix.t())
        aggr_out = self.sigmoid(aggr_out)
        return aggr_out
    def calculate_weights(self,index):
        """
        计算每个索引位置的权重（根据相同元素的数量）
        Args:
            index: 索引数组，如 [0,0,0,1,1,2,2,3]
        Returns:
            weights: 权重数组，如 [0.33,0.33,0.33,0.5,0.5,0.5,0.5,1]
        """
        # 使用 unique_counts 获取每个元素的计数
        unique_elements, counts = torch.unique(index, return_counts=True)
        
        # 创建一个和输入相同大小的权重张量
        weights = torch.zeros_like(index, dtype=torch.float)
        
        # 为每个元素分配权重
        for element, count in zip(unique_elements, counts):
            weights[index == element] = 1.0 / count.float()
        
        return weights

In [15]:
model = GNN(user_x.size(dim=1),user_x.size(dim=1))
model = model.to(device)
data = data.to(device)
nn_data = nn_data.to(device)


In [ ]:

emb = model.forward(nn_data)
print(emb.size())

In [17]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """
    Args:
        users_emb_final (torch.Tensor): e_u^k
        users_emb_0 (torch.Tensor): e_u^0
        pos_items_emb_final (torch.Tensor): positive e_i^k
        pos_items_emb_0 (torch.Tensor): positive e_i^0
        neg_items_emb_final (torch.Tensor): negative e_i^k
        neg_items_emb_0 (torch.Tensor): negative e_i^0
        lambda_val (float): λ的值
    Returns:
        torch.Tensor: loss值
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2))  # L2 loss L2范数是指向量各元素的平方和然后求平方根

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1) # 正采样预测分数
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1) # 负采样预测分数

    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss

    return loss

In [19]:
def lossfunction(emd_out,data: HeteroData):
        x_src = data['src']
        edge_index = data['adj']
        x_src = x_src[edge_index[0]]
        x_src = x_src.t()
        simliarity = torch.matmul(emd_out , x_src)
        simliarity = torch.sum(simliarity, dim=0)
        loss = torch.mean(simliarity)
        return loss

In [21]:
def sample_items(items_emb, num_samples):
    indices = torch.randperm(items_emb.size(0))[:num_samples]
    return items_emb[indices]

In [22]:
learning_rate = 1e-1
batch_size = 64
iteration = 5000
lambda_val = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
losses = []

In [ ]:
for iter in trange(iteration):
    users_emb = model.forward(nn_data)
    edges = structured_negative_sampling(nn_data['adj'])
    edges = torch.stack(edges, dim=0)
    neg_data = nn_data['src']
    neg_data = neg_data[edges[2]]
    loss = lossfunction(users_emb,nn_data)
    num_users = users_emb.size(0)
    pos_items_emb_final = sample_items(nn_data['src'], num_users)
    neg_items_emb_final = sample_items(neg_data, num_users)
    bpr_loss_value = bpr_loss(users_emb,nn_data['tgt'],pos_items_emb_final,pos_items_emb_final,
                              neg_items_emb_final,neg_items_emb_final,lambda_val)
    # 记录损失
    losses.append(bpr_loss_value.item())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if iter % 100 == 0:
        print("loss = ",bpr_loss_value)
print("Train done !")


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('Training Loss Over Time')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid(True)
plt.show()
